In [1]:
import sys
import os
import cv2
import importlib
import torch
import numpy as np
import argparse
import yaml
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageOps
import json
import nltk
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import clip
import kornia
import torchvision
import skimage.color
import random

from html_images import *
from sample_func import *

In [2]:
def getImage(path):
    for ext in ['jpg', 'png', 'JPEG']:
        file = path + '.' + ext
        if os.path.isfile(file):
            return Image.open(file).convert('RGB')

    return None


def resize_ab(org, img):
    img = img.resize(org.size)

    org = np.array(org)
    l = cv2.cvtColor(org, cv2.COLOR_RGB2LAB)[:, :, 0:1]

    img = np.array(img)
    ab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)[:, :, 1:3]

    lab = np.concatenate([l, ab], axis=2)
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    return Image.fromarray(rgb)

# Visualize raw

In [ ]:
dataset = 'imagenet'

save_dir = './result/random100'
orig_dir = 'original'
comp_dirs = ['instcolorization', 'deoldify', 'coltran', 'ours/sample_resize']
num = 50

# Save result in HTML file
html = HTML(save_dir, 'Sample')
html.add_header(save_dir)

files = os.listdir(os.path.join(dataset, orig_dir))
np.random.shuffle(files)

for i, f in tqdm(enumerate(files[:num])):
    images = []

    fname = f.split('.')[0]
    org_img = getImage(os.path.join(dataset, orig_dir, fname))
    images.append(org_img.convert('L'))
    images.append(org_img.copy())

    for path in comp_dirs:
        img = getImage(os.path.join(dataset, path, fname))
        if img == None:
            img = Image.new('RGB', org_img.size)
        if list(img.size) == [256, 256]:
            img = resize_ab(org_img, img)
        images.append(img.copy())
    
    text = comp_dirs.copy()
    text.insert(0, orig_dir)
    text[-1] += ' ' + f
    save_result(html, index=i, images=images, texts=text)
    html.save()